## 5. Tool use(crewai)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [2]:
import os
from crewai import Agent, Task, Crew
from crewai.tools import tool
import logging

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### 1. 도구 정의

In [4]:
# 이 도구는 이제 원시 데이터(부동소수점)를 반환하거나 표준 파이썬 예외를 발생시킵니다.
# 이렇게 하면 재사용성이 높아지고, 에이전트가 결과를 적절히 처리하도록 강제합니다.

@tool("주가 조회 도구")
def get_stock_price(ticker: str) -> float:
    """
    지정한 주식 티커 심볼의 최신(모의) 주가를 가져옵니다.
    가격을 float으로 반환합니다. 티커를 찾을 수 없으면 ValueError를 발생시킵니다.
    """
    logging.info(f"도구 호출: get_stock_price | 티커 '{ticker}'")
    simulated_prices = {
        "AAPL": 178.15,
        "GOOGL": 1750.30,
        "MSFT": 425.50,
    }
    price = simulated_prices.get(ticker.upper())
    if price is not None:
        return price
    else:
        # 문자열을 반환하는 것보다 구체적인 예외를 발생시키는 편이 좋습니다.
        # 에이전트는 예외를 처리할 수 있으며 다음 동작을 스스로 결정할 수 있습니다.
        raise ValueError(f"티커 '{ticker.upper()}'에 대한 모의 가격을 찾을 수 없습니다.")

### 2. Agent 정의

In [5]:
financial_analyst_agent = Agent(
  role='수석 재무 분석가',
  goal='제공된 도구를 사용해 주식 데이터를 분석하고 핵심 가격을 보고합니다.',
  backstory="당신은 데이터 소스를 활용해 주식 정보를 찾는 데 능숙한 숙련된 재무 분석가입니다. 명확하고 직설적인 답변을 제공합니다.",
  verbose=True,
  tools=[get_stock_price],
  # 위임을 허용하는 것이 유용할 수 있지만, 이 간단한 작업에는 필수는 아닙니다.
  allow_delegation=False,
)

### 3. Task 정의

In [6]:
# 작업 설명이 더 구체적이며, 데이터 조회 성공/실패에 대한 에이전트의 반응을 안내합니다.
analyze_aapl_task = Task(
  description="""주가 조회 도구(Stock Price Lookup Tool)를 사용하여 AAPL 주식의 현재 가격을 확인하세요.
  도구 호출이 성공하면 가격을 명확하게 보고하세요.
  도구가 오류를 발생시키면, 해당 티커를 찾을 수 없음을 설명하고 대안을 제시하세요.
  """,
  expected_output="주가에 대한 명확한 보고 또는 오류 설명.",
  agent=financial_analyst_agent,
)

### 4. Crew 구성

In [7]:
financial_crew = Crew(
  agents=[financial_analyst_agent],
  tasks=[analyze_aapl_task],
  verbose=True  # 프로덕션에서는 상세 로그를 줄이려면 False로 설정
)

### 5. 실행

In [8]:
def main():
    """크루를 실행하는 메인 함수."""
    # 런타임 오류를 피하기 위해 시작 전에 API 키를 확인합니다.
    if not os.environ.get("OPENAI_API_KEY"):
        print("ERROR: OPENAI_API_KEY 환경 변수가 설정되어 있지 않습니다.")
        print("스크립트를 실행하기 전에 설정해주세요.")
        return
    print("\n## Financial 크루 시작...")
    print("---------------------------------")
    # kickoff 메서드는 실행을 시작합니다.
    result = financial_crew.kickoff()
    print("\n---------------------------------")
    print("## 크루 실행이 완료되었습니다.")
    print("\n최종 결과:\n", result)

if __name__ == "__main__":
    main()

10:56:13 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-4o; provider = openai
2025-10-15 10:56:13,537 - INFO - 
LiteLLM completion() model= gpt-4o; provider = openai



## Financial 크루 시작...
---------------------------------
# Agent: 수석 재무 분석가
## Task: 주가 조회 도구(Stock Price Lookup Tool)를 사용하여 AAPL 주식의 현재 가격을 확인하세요.
  도구 호출이 성공하면 가격을 명확하게 보고하세요.
  도구가 오류를 발생시키면, 해당 티커를 찾을 수 없음을 설명하고 대안을 제시하세요.
  


10:56:15 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
2025-10-15 10:56:15,022 - INFO - Wrapper: Completed Call, calling success_handler
2025-10-15 10:56:15,032 - INFO - 도구 호출: get_stock_price | 티커 'AAPL'
10:56:15 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-4o; provider = openai
2025-10-15 10:56:15,033 - INFO - 
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: 수석 재무 분석가
## Using tool: 주가 조회 도구
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
178.15


10:56:16 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
2025-10-15 10:56:16,082 - INFO - Wrapper: Completed Call, calling success_handler




# Agent: 수석 재무 분석가
## Final Answer: 
AAPL 주식의 현재 가격은 $178.15입니다.



---------------------------------
## 크루 실행이 완료되었습니다.

최종 결과:
 AAPL 주식의 현재 가격은 $178.15입니다.
